# Ground Truth Evaluations

In this quickstart you will create a evaluate a LangChain app using ground truth. Ground truth evaluation can be especially useful during early LLM experiments when you have a small set of example queries that are critical to get right.

Ground truth evaluation works by comparing the similarity of an LLM response compared to its matching verified response.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/expositional/frameworks/langchain/langchain_groundtruth.ipynb)

### Import from LlamaIndex and TruLens

In [ ]:
# ! pip install trulens_eval==0.18.1 langchain>=0.0.342

In [ ]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.prompts import HumanMessagePromptTemplate

from trulens_eval import Feedback, Tru, TruChain
from trulens_eval.feedback import GroundTruthAgreement, Huggingface
tru = Tru()

### Add API keys
For this quickstart, you will need Open AI keys.

In [ ]:
import os
os.environ["HUGGINGFACE_API_KEY"] = "hf_..."
os.environ["OPENAI_API_KEY"] = "sk-..."

### Create Simple LLM Application

This example uses Langchain with an OpenAI LLM.

In [ ]:
full_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=
        "Provide an answer to the following: {prompt}",
        input_variables=["prompt"],
    )
)

chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

llm = OpenAI(temperature=0.9, max_tokens=128)

chain = LLMChain(llm=llm, prompt=chat_prompt_template, verbose=True)

## Initialize Feedback Function(s)

In [ ]:
golden_set = [
    {"query": "who invented the lightbulb?", "response": "Thomas Edison"},
    {"query": "¿quien invento la bombilla?", "response": "Thomas Edison"}
]

f_groundtruth = Feedback(GroundTruthAgreement(golden_set).agreement_measure, name = "Ground Truth").on_input_output()

# Define a language match feedback function using HuggingFace.
hugs = Huggingface()
f_lang_match = Feedback(hugs.language_match).on_input_output()

## Instrument chain for logging with TruLens

In [ ]:
tc = TruChain(chain, feedbacks=[f_groundtruth, f_lang_match])

In [ ]:
# Instrumented query engine can operate as a context manager:
with tc as recording:
    chain("¿quien invento la bombilla?")
    chain("who invented the lightbulb?")

## Explore in a Dashboard

In [ ]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed